In [2]:
%%capture
!pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import polara
from polara import get_movielens_data
from polara.preprocessing.dataframes import leave_one_out, reindex

from dataprep import transform_indices
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

from polara.lib.tensor import hooi
from polara.lib.sparse import tensor_outer_at
from polara.evaluation.pipelines import random_grid

from sa_hooi import sa_hooi, form_attention_matrix, get_scaling_weights, generate_position_projector

from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import norm, svds

from IPython.utils import io

# Data preparation

In [2]:
data = get_movielens_data('ml-1m.zip', include_time=True)

## Data splitting

In [3]:
test_timepoint = data['timestamp'].quantile(
    q=0.9, interpolation='nearest'
)

In [4]:
test_data_ = data.query('timestamp >= @test_timepoint')

In [5]:
test_data_.nunique()

userid        1209
movieid       3407
rating           5
timestamp    63411
dtype: int64

In [6]:
train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
)

In [7]:
training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')

In [8]:
test_data = reindex(test_data_, data_index['items'])

Filtered 113 invalid observations.


In [9]:
test_data.nunique()

userid        1208
movieid       3365
rating           5
timestamp    63322
dtype: int64

In [10]:
testset_, holdout_ = leave_one_out(
    test_data, target='timestamp', sample_top=True, random_state=0
)
testset_valid_, holdout_valid_ = leave_one_out(
    testset_, target='timestamp', sample_top=True, random_state=0
)

In [11]:
test_users = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
testset_valid = testset_valid_.query('userid in @test_users').sort_values('userid')
holdout_valid = holdout_valid_.query('userid in @test_users').sort_values('userid')

In [12]:
testset_valid.nunique()

userid        1137
movieid       3357
rating           5
timestamp    61488
dtype: int64

In [13]:
assert holdout_valid.set_index('userid')['timestamp'].ge(
    testset_valid
    .groupby('userid')
    ['timestamp'].max()
).all()

In [14]:
data_description = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    n_ratings = training['rating'].nunique(),
    min_rating = training['rating'].min(),
    test_users = holdout_valid[data_index['users'].name].drop_duplicates().values,
    n_test_users = holdout_valid[data_index['users'].name].nunique()
)

In [15]:
def make_prediction(tf_scores, holdout_valid, data_description):
    #print("3 + 4 + 5 - 2 - 1:")
    #print("3 + 4 + 5:")
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, mrr, cov = model_evaluate(tf_recs, holdout_valid, data_description, topn=n)
        print(f"Validation : HR@{n} = {hr:.4f}, MRR@{n} == {mrr:.4f}, Coverage@{n} = {cov:.4f}")

In [16]:
def valid_mlrank(mlrank):
    '''
    Only allow ranks that are suitable for truncated SVD computations
    on unfolded compressed tensor (the result of ttm product in HOOI).
    '''
    r1, r2, r3 = mlrank
    return r1*r2 > r3 and r1*r3 > r2 and r2*r3 > r1

# Sequential TF

## Preprocessing

In [17]:
n_pos = 200

def assign_positions(s, maxlen=n_pos):
    return np.arange(maxlen-len(s), maxlen)

training_data = (
    training
    .sort_values('timestamp')
    .assign(
        pos = lambda df: df.groupby('userid')['movieid'].transform(assign_positions)
    )
    .sort_values(['userid', 'timestamp'])
    .query('pos>=0')
)

testset_valid = (
    testset_valid
    .sort_values('timestamp')
    .assign(
        pos = lambda df: df.groupby('userid')['movieid'].transform(assign_positions)
    )
    .sort_values(['userid', 'timestamp'])
)

## Model

In [18]:
data_description_seq = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    positions = 'pos',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    n_pos = n_pos
)
data_description

{'users': 'userid',
 'items': 'movieid',
 'feedback': 'rating',
 'n_users': 4831,
 'n_items': 3635,
 'n_ratings': 5,
 'min_rating': 1,
 'test_users': array([   1,    2,    3, ..., 6002, 6016, 6040], dtype=int64),
 'n_test_users': 1137}

In [19]:
def seqtf_model_build(config, data, data_description):
    userid = data_description["users"]
    itemid = data_description["items"]
    positions = data_description["positions"]

    n_users = data_description["n_users"]
    n_items = data_description["n_items"]
    max_pos = data_description["n_pos"]
    shape = (n_users, n_items, max_pos)

    attention_matrix = form_attention_matrix(
        data_description["n_pos"],
        config["attention_decay"],
        format = 'csr'
    )

    item_popularity = (
        data[itemid]
        .value_counts(sort=False)
        .reindex(range(n_items))
        .fillna(1)
        .values
    )
    scaling_weights = get_scaling_weights(item_popularity, scaling=config["scaling"])

    idx = data[[userid, itemid, positions]].values
    val = np.ones(idx.shape[0], dtype='f8')

    user_factors, item_factors, feedback_factors = sa_hooi(
        idx, val, shape, config["mlrank"],
        attention_matrix = attention_matrix,
        scaling_weights = scaling_weights,
        max_iters = config["num_iters"],
        parallel_ttm = False,
        randomized = config["randomized"],
        growth_tol = config["growth_tol"],
        seed = config["seed"],
        iter_callback = None,
    )
    return user_factors, item_factors, feedback_factors, attention_matrix


In [20]:
config = {
    "scaling": 1,
    "mlrank": (30, 30, 5),
    "n_pos": n_pos,
    "num_iters": 5,
    "attention_decay": 1,
    "randomized": True,
    "growth_tol": 1e-4,
    "seed": 42
}

In [21]:
def tf_scoring(params, data, data_description):
    user_factors, item_factors, pos_factors, attention_matrix = params
    last_position_projector = generate_position_projector(attention_matrix, pos_factors)

    userid = data_description["users"]
    itemid = data_description["items"]
    posid = data_description["positions"]

    tset_data = data.sort_values([userid, posid])
    useridx = tset_data[userid].values
    itemidx = tset_data[itemid].values
    indptr, = np.where(np.diff(useridx, prepend=0, append=1))
    scores = user_scoring(indptr, itemidx, item_factors, last_position_projector)
    return scores

def user_scoring(indptr, indices, item_factors, last_position_projector):
    sequences = np.array_split(indices, indptr[1:-1])
    n_items = item_factors.shape[0]
    scores = np.zeros((len(sequences), n_items))
    for u, seq in enumerate(sequences):
        scores[u] = sequences_score(seq, item_factors, last_position_projector)
    return scores

def sequences_score(seq, item_factors, last_position_projector):
    n_pos = len(last_position_projector)
    user_profile = item_factors[seq[-(n_pos-1):], :]
    n_items = user_profile.shape[0]
    scores = item_factors @ (user_profile.T @ last_position_projector[-(n_items+1):-1])
    return scores

## Tuning

In [24]:
tf_hyper = { # hyper-parameters dict for multi-linear rank
    'r1': range(20, 80, 10),
    'r2': range(20, 80, 10),
    'r3': range(5, 40, 5),
}

grid, param_names = random_grid(tf_hyper, n=0)
tf_grid = [tuple(mlrank) for mlrank in grid if valid_mlrank(mlrank)]

In [ ]:
hr_tf = {}
mrr_tf = {}
cov_tf = {}
for mlrank in tqdm(tf_grid):
    with io.capture_output() as captured:
        config['mlrank'] = mlrank
        tf_params = seqtf_model_build(config, training_data, data_description)
        tf_scores = tf_scoring(tf_params, testset_valid, data_description)
        downvote_seen_items(tf_scores, testset_valid, data_description)
        tf_recs = topn_recommendations(tf_scores, topn=10) # comlete the code
        hr, mrr, cov = model_evaluate(tf_recs, holdout_valid, data_description, topn=10)
        #hr, mrr, cov = evaluate(tf_recs, valid_holdout, data_description)
        hr_tf[mlrank] = hr
        mrr_tf[mlrank] = mrr
        cov_tf[mlrank] = cov

  0%|          | 0/252 [00:00<?, ?it/s]

In [ ]:
print(f'Best HR={pd.Series(hr_tf).max():.4f} achieved with mlrank={pd.Series(hr_tf).idxmax()}')
print(f'Best MRR={pd.Series(mrr_tf).max():.4f} achieved with mlrank={pd.Series(mrr_tf).idxmax()}')
print(f'COV={pd.Series(cov_tf)[pd.Series(hr_tf).idxmax()]:.4f} (based on best HR value)')

In [ ]:
print(f'COV={pd.Series(cov_tf)[pd.Series(hr_tf).idxmax()]:.4f} (based on best HR value)')

# Random Model

In [22]:
def build_random_model(trainset, trainset_description):
    itemid = trainset_description['items']
    n_items = trainset[itemid].max() + 1
    random_state = np.random.RandomState(42)
    return n_items, random_state

def random_model_scoring(params, testset, testset_description):
    n_items, random_state = params
    n_users = testset_description['n_test_users']
    scores = random_state.rand(n_users, n_items)
    return scores

def simple_model_recom_func(scores, topn=20):
    recommendations = np.apply_along_axis(topidx, 1, scores, topn)
    return recommendations

def topidx(a, topn):
    parted = np.argpartition(a, -topn)[-topn:]
    return parted[np.argsort(-a[parted])]

# Popularity-based model

In [23]:
def build_popularity_model(trainset, trainset_description):
    itemid = trainset_description['items']
    item_popularity = trainset[itemid].value_counts()
    return item_popularity

def popularity_model_scoring(params, testset, testset_description):
    item_popularity = params
    n_items = item_popularity.index.max() + 1
    n_users = testset_description['n_test_users']
    # fill in popularity scores for each item with indices from 0 to n_items-1
    popularity_scores = np.zeros(n_items,)
    popularity_scores[item_popularity.index] = item_popularity.values
    # same scores for each test user
    scores = np.tile(popularity_scores, n_users).reshape(n_users, n_items)
    return scores

# PureSVD

In [24]:
def matrix_from_observations(data, data_description):
    useridx = data[data_description['users']]
    itemidx = data[data_description['items']]
    values = data[data_description['feedback']]
    return csr_matrix((values, (useridx, itemidx)), dtype='f8')


def build_svd_model(config, data, data_description):
    source_matrix = matrix_from_observations(data, data_description)
    D = norm(source_matrix, axis=0)
    A = source_matrix.dot(diags(D**(config['f']-1)))
    _, _, vt = svds(A, k=config['rank'], return_singular_vectors='vh')
#     singular_values = s[::-1]
    item_factors = np.ascontiguousarray(vt[::-1, :].T)
    return item_factors

def svd_model_scoring(params, data, data_description):
    item_factors = params
    test_data = data.assign(
        userid = pd.factorize(data['userid'])[0]
    )
    test_matrix = matrix_from_observations(test_data, data_description)
    scores = test_matrix.dot(item_factors) @ item_factors.T
    return scores

## Tuning

In [76]:
rank_grid = np.arange(20, 300, 10)
f_grid = np.linspace(-2, 2, 21)

In [79]:
hr_tf = {}
mrr_tf = {}
cov_tf = {}
grid = list(zip(np.meshgrid(rank_grid, f_grid)[0].flatten(), np.meshgrid(rank_grid, f_grid)[1].flatten()))
for params in tqdm(grid):
    r, f = params
    svd_config = {'rank': int(r), 'f': f}
    svd_params = build_svd_model(svd_config, training, data_description)
    svd_scores = svd_model_scoring(svd_params, testset_valid, data_description)
    downvote_seen_items(svd_scores, testset_valid, data_description)
    svd_recs = topn_recommendations(svd_scores, topn=10)
    hr, mrr, cov = model_evaluate(svd_recs, holdout_valid, data_description, topn=10)
    hr_tf[f'r={r}, f={f:.2f}'] = hr
    mrr_tf[f'r={r}, f={f:.2f}'] = mrr
    cov_tf[f'r={r}, f={f:.2f}'] = cov

  0%|          | 0/588 [00:00<?, ?it/s]

In [80]:
hr_sorted = sorted(hr_tf, key=hr_tf.get, reverse=True)
for i in range(10):
    print(hr_sorted[i], hr_tf[hr_sorted[i]])

r=230, f=0.80 0.10026385224274406
r=290, f=0.40 0.09762532981530343
r=120, f=1.00 0.09762532981530343
r=220, f=0.40 0.09674582233948989
r=230, f=0.40 0.09674582233948989
r=250, f=0.40 0.09674582233948989
r=170, f=0.60 0.09674582233948989
r=240, f=0.60 0.09674582233948989
r=250, f=0.60 0.09674582233948989
r=120, f=1.20 0.09674582233948989


In [83]:
mrr_sorted = sorted(mrr_tf, key=mrr_tf.get, reverse=True)
for i in range(10):
    print(mrr_sorted[i], mrr_tf[mrr_sorted[i]])

r=200, f=0.80 0.03967172034454357
r=230, f=0.60 0.039258491435272436
r=280, f=0.80 0.039107718725132974
r=280, f=0.60 0.03910353059429578
r=290, f=0.60 0.03900231743239659
r=250, f=0.80 0.038775809356284295
r=220, f=0.60 0.03843308064943949
r=230, f=0.80 0.03839119934106742
r=170, f=0.40 0.03828544903742793
r=290, f=0.80 0.03825543409976129


In [82]:
cov_sorted = sorted(cov_tf, key=cov_tf.get, reverse=True)
for i in range(10):
    print(cov_sorted[i], cov_tf[cov_sorted[i]])

r=290, f=-0.40 0.5026134800550206
r=280, f=-0.40 0.49793672627235214
r=280, f=-0.20 0.4968363136176066
r=290, f=-0.20 0.49436038514442915
r=270, f=-0.40 0.49353507565337
r=270, f=-0.20 0.4921595598349381
r=260, f=-0.20 0.4905089408528198
r=240, f=-0.20 0.48830811554332876
r=250, f=-0.20 0.48830811554332876
r=260, f=-0.40 0.48775790921595596


# Test metrics

In [25]:
def model_evaluate(recommended_items, holdout, holdout_description, alpha=3, topn=10):
    itemid = holdout_description['items']
    rateid = holdout_description['feedback']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    # HR calculation
    hr = np.mean(hits_mask.any(axis=1))
    # MRR calculation
    n_test_users = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_users
    # DCG calculation
    pos_mask = (holdout[rateid] >= alpha).values
    neg_mask = (holdout[rateid] < alpha).values
    pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
    neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
    ndcg = np.mean(1 / np.log2(pos_hit_rank+1))
    ndcl = np.mean(1 / np.log2(neg_hit_rank+1))
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = np.unique(recommended_items).size / n_items
    return hr, mrr, cov, ndcg, ndcl

In [26]:
def make_prediction(tf_scores, holdout_valid, data_description):
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, mrr, cov, dcg, dcl = model_evaluate(tf_recs, holdout_valid, data_description, topn=n)
        print(f"Test : HR@{n} = {hr:.4f}, MRR@{n} = {mrr:.4f}, Coverage@{n} = {cov:.4f}, nDCG@{n} = {dcg}, nDCL@{n} = {dcl}")

In [27]:
test_users = np.intersect1d(testset_.userid.unique(), holdout_.userid.unique())
testset = testset_.query('userid in @test_users').sort_values('userid')
holdout = holdout_.query('userid in @test_users').sort_values('userid')

In [28]:
data_description = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    n_ratings = training['rating'].nunique(),
    min_rating = training['rating'].min(),
    test_users = holdout[data_index['users'].name].drop_duplicates().values,
    n_test_users = holdout[data_index['users'].name].nunique()
)

## Random model

In [30]:
rnd_params = build_random_model(training, data_description)
rnd_scores = random_model_scoring(rnd_params, None, data_description)
downvote_seen_items(rnd_scores, testset, data_description)

make_prediction(rnd_scores, holdout, data_description)

Test : HR@5 = 0.0009, MRR@5 = 0.0004, Coverage@5 = 0.7948, nDCG@5 = 0.6309297535714575, nDCL@5 = nan
Test : HR@10 = 0.0017, MRR@10 = 0.0005, Coverage@10 = 0.9585, nDCG@10 = 0.4599972899446727, nDCL@10 = nan
Test : HR@20 = 0.0068, MRR@20 = 0.0009, Coverage@20 = 0.9983, nDCG@20 = 0.32322642650610806, nDCL@20 = 0.2550253104977256


C:\Users\Liza\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Liza\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Popularity-based model

In [31]:
pop_params = build_popularity_model(training, data_description)
pop_scores = popularity_model_scoring(pop_params, None, data_description)
downvote_seen_items(pop_scores, testset, data_description)

In [32]:
make_prediction(pop_scores, holdout, data_description)

Test : HR@5 = 0.0051, MRR@5 = 0.0034, Coverage@5 = 0.0099, nDCG@5 = 0.8, nDCL@5 = 0.43067655807339306
Test : HR@10 = 0.0111, MRR@10 = 0.0041, Coverage@10 = 0.0165, nDCG@10 = 0.5161361678203137, nDCL@10 = 0.43067655807339306
Test : HR@20 = 0.0291, MRR@20 = 0.0053, Coverage@20 = 0.0292, nDCG@20 = 0.35043323231744705, nDCL@20 = 0.3504573562503564


## PureSVD

In [33]:
svd_config = {'rank': 200, 'f': 0.8}
svd_params = build_svd_model(svd_config, training, data_description)
svd_scores = svd_model_scoring(svd_params, testset, data_description)
downvote_seen_items(svd_scores, testset, data_description)

make_prediction(svd_scores, holdout, data_description)

Test : HR@5 = 0.0522, MRR@5 = 0.0271, Coverage@5 = 0.2006, nDCG@5 = 0.6309242785123235, nDCL@5 = 0.681057389859924
Test : HR@10 = 0.0753, MRR@10 = 0.0302, Coverage@10 = 0.2644, nDCG@10 = 0.5427472913702939, nDCL@10 = 0.5321384513866659
Test : HR@20 = 0.1241, MRR@20 = 0.0335, Coverage@20 = 0.3403, nDCG@20 = 0.426638518802019, nDCL@20 = 0.4278542521468115


## SeqTF

In [34]:
testset = (
    testset
    .sort_values('timestamp')
    .assign(
        pos = lambda df: df.groupby('userid')['movieid'].transform(assign_positions)
    )
    .sort_values(['userid', 'timestamp'])
)

In [35]:
data_description_seq = dict(
    users = data_index['users'].name,
    items = data_index['items'].name,
    feedback = 'rating',
    positions = 'pos',
    n_users = len(data_index['users']),
    n_items = len(data_index['items']),
    n_pos = n_pos,
    test_users = holdout[data_index['users'].name].drop_duplicates().values,
    n_test_users = holdout[data_index['users'].name].nunique()
)
data_description

{'users': 'userid',
 'items': 'movieid',
 'feedback': 'rating',
 'n_users': 4831,
 'n_items': 3635,
 'n_ratings': 5,
 'min_rating': 1,
 'test_users': array([   1,    2,    3, ..., 6002, 6016, 6040], dtype=int64),
 'n_test_users': 1168}

In [36]:
config['mlrank'] = (30, 70, 5)
config

{'scaling': 1,
 'mlrank': (30, 70, 5),
 'n_pos': 200,
 'num_iters': 5,
 'attention_decay': 1,
 'randomized': True,
 'growth_tol': 0.0001,
 'seed': 42}

In [37]:
tf_params = seqtf_model_build(config, training_data, data_description_seq)
tf_scores = tf_scoring(tf_params, testset, data_description_seq)
downvote_seen_items(tf_scores, testset, data_description_seq)

make_prediction(tf_scores, holdout, data_description_seq)

growth of the core: 1.0
growth of the core: 0.3032713941930027
growth of the core: 0.04349301012106686
growth of the core: 0.00561007448836323
growth of the core: 0.0017265875391681215
Test : HR@5 = 0.0479, MRR@5 = 0.0264, Coverage@5 = 0.1615, nDCG@5 = 0.6668021542037992, nDCL@5 = 0.6262581788155304
Test : HR@10 = 0.0822, MRR@10 = 0.0308, Coverage@10 = 0.2182, nDCG@10 = 0.5295437826196445, nDCL@10 = 0.46811407420090356
Test : HR@20 = 0.1293, MRR@20 = 0.0341, Coverage@20 = 0.2933, nDCG@20 = 0.4327447755392594, nDCL@20 = 0.37725236118946887


C:\Users\Liza\anaconda3\lib\site-packages\scipy\sparse\coo.py:431: SparseEfficiencyWarning: Constructing a DIA matrix with 200 diagonals is inefficient
  warn("Constructing a DIA matrix with %d diagonals "
